### Урок 3. Embedding word2vec fasttext

Задача поиск похожих по эмбеддингам

Скачиваем датасет (источник): положительные, отрицательные.
или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

что надо сделать:
объединить в одну выборку
на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов (на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)

Проверить насколько хорошо работают подходы

In [22]:
!pip install pymorphy2
!pip install annoy
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [2]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
df = positive.append(negative)

In [3]:
texts = list(df['text'])

In [4]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [5]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

for text in texts:
    text = preprocess_txt(text)
    sentences.append(text)
    c += 1


In [6]:
modelW2V = Word2Vec(sentences=sentences, size=300, window=5, min_count=1)

In [7]:
modelFT = FastText(sentences=sentences, size=300, min_count=1, window=5)

In [8]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

In [9]:
index_map = {}
counter = 0

for text in tqdm_notebook(texts):
        n_w2v = 0
        n_ft = 0
        index_map[counter] = text
        question = preprocess_txt(text)
        
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        


w2v_index.build(10)
ft_index.build(10)

<ipython-input-9-ac90b1976197>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for text in tqdm_notebook(texts):


  0%|          | 0/226834 [00:00<?, ?it/s]

True

In [10]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3, )
    return [index_map[i] for i in answers]

In [14]:
TEXT = "купить покушать пельмени"

In [15]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['@uawtriannocxs я жестко засрала этот магаз но бомбочку я там все же куплю)',
 'съездила  в Галилео , ничерта не купила :(',
 'И если я куплю это все,то 105% съезжу в этот гребанный Шерегеш)))']

In [16]:
get_response(TEXT, ft_index, modelFT, index_map)

['Мда, где бы его еще купить, если скачать уже нельзя?:)',
 'Пошли купим воды. Нет , я уже не хочу убери её от меня)',
 'Думала куплю себе платьев в поездке, какой там( ни одного не купила']